In [1]:
#Jai Sri Ram

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import * 

spark = SparkSession.builder \
    .appName("Financial_Risk") \
    .getOrCreate()


In [3]:
print("Spark is ready:", spark.version)

Spark is ready: 3.5.0


In [4]:
spark.version

'3.5.0'

In [5]:
import boto3
s3 = boto3.client("s3")
s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'ACK1K6500PGEM92B',
  'HostId': 'yjdi/D0ZgTt8XSELRmepVtwQxdB5gEJaw44vPwd+9GaID+KEy6Fc033WpDAqIMR+HPGHh+5IcXg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'yjdi/D0ZgTt8XSELRmepVtwQxdB5gEJaw44vPwd+9GaID+KEy6Fc033WpDAqIMR+HPGHh+5IcXg=',
   'x-amz-request-id': 'ACK1K6500PGEM92B',
   'date': 'Sun, 03 Aug 2025 07:46:30 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aws-logs-315689562130-us-east-1',
   'CreationDate': datetime.datetime(2024, 12, 29, 11, 7, 52, tzinfo=tzlocal())},
  {'Name': 'ec2-emr-prod-data',
   'CreationDate': datetime.datetime(2024, 12, 29, 10, 19, 48, tzinfo=tzlocal())},
  {'Name': 'ec2-emr-raw-data',
   'CreationDate': datetime.datetime(2024, 12, 29, 10, 19, 14, tzinfo=tzlocal())},
  {'Name': 'emr-ec2-processing-script',
   'CreationDate': datetime.datetime(2024, 12, 29, 10, 34, 23, tzinfo=tzlocal())},
  {'Name': 'my-a

In [6]:
df = spark.read.csv('financial_risk.csv',header=True, inferSchema = True)

In [7]:
df.show()

+---+----------+---------------+--------------+--------+------------+-----------+------------+-----------------+--------------------+---------------+--------------------+------------+--------------------+----------------+-----+--------------------+-----------------+---------------------+-----------+
|Age|    Gender|Education Level|Marital Status|  Income|Credit Score|Loan Amount|Loan Purpose|Employment Status|Years at Current Job|Payment History|Debt-to-Income Ratio|Assets Value|Number of Dependents|            City|State|             Country|Previous Defaults|Marital Status Change|Risk Rating|
+---+----------+---------------+--------------+--------+------------+-----------+------------+-----------------+--------------------+---------------+--------------------+------------+--------------------+----------------+-----+--------------------+-----------------+---------------------+-----------+
| 49|      Male|            PhD|      Divorced| 72799.0|       688.0|    45713.0|    Business|   

In [8]:
df_main = df.selectExpr("row_number() over(order by age) as id", "*")

In [10]:
df_main.show(9)

+---+---+----------+---------------+--------------+--------+------------+-----------+------------+-----------------+--------------------+---------------+--------------------+------------+--------------------+-----------------+-----+--------------------+-----------------+---------------------+-----------+
| id|Age|    Gender|Education Level|Marital Status|  Income|Credit Score|Loan Amount|Loan Purpose|Employment Status|Years at Current Job|Payment History|Debt-to-Income Ratio|Assets Value|Number of Dependents|             City|State|             Country|Previous Defaults|Marital Status Change|Risk Rating|
+---+---+----------+---------------+--------------+--------+------------+-----------+------------+-----------------+--------------------+---------------+--------------------+------------+--------------------+-----------------+-----+--------------------+-----------------+---------------------+-----------+
|  1| 18|    Female|            PhD|        Single| 87346.0|        NULL|    26863

In [11]:
df_main_1 = df_main.selectExpr("id","Gender",
    "`Education level` as Qualification",
    "`Marital Status` as Marital_status",
    "Income",
    "`Credit Score` as Credit_score",
    "`Loan Amount` as Loan_Amount",
    "`Loan Purpose` as Loan_Purpose",
    "`Employment Status` as Employment_Status",
    "`Years at Current Job` as Current_Job_YOE",
    "`Payment History` as Payment_History",
    "`Debt-to-Income Ratio` as Debt_to_Income_Ratio",
    "`Assets Value` as Assets_Value",
    "`Number of Dependents` as Dependents",
    "City",
    "State",
    "Country",
    "`Previous Defaults` as Previous_Defaults",
    "`Risk Rating` as Risk_Rating"
)


In [12]:
df_main_1.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|   Bachelor's|       Widowed| 25605.0|       751.0|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|       3.0|      Samanthastad|  

In [13]:
df_gender = df_main_1.select('Gender').groupBy('Gender').count()

In [14]:
df_gender.show()

+----------+-----+
|    Gender|count|
+----------+-----+
|    Female| 4990|
|Non-binary| 5059|
|      Male| 4951|
+----------+-----+



In [15]:
df_main_1.createOrReplaceTempView('financial_risk_table')

In [16]:
df_loanPurpose = spark.sql("select Loan_Purpose,count(1) from financial_risk_table group by Loan_Purpose")

In [17]:
df_loanPurpose.show()

+------------+--------+
|Loan_Purpose|count(1)|
+------------+--------+
|        Home|    3766|
|    Personal|    3771|
|        Auto|    3725|
|    Business|    3738|
+------------+--------+



In [18]:
df_study = spark.sql("select qualification,count(1) cnt from financial_risk_table group by 1")

In [19]:
df_study.show()

+-------------+----+
|qualification| cnt|
+-------------+----+
|  High School|3774|
|          PhD|3760|
|   Bachelor's|3829|
|     Master's|3637|
+-------------+----+



In [20]:
df_qualification_altered = spark.sql("""
    SELECT 
        CASE 
            WHEN Trim(Qualification) LIKE '%Bachelor''s%' THEN 'UG'
            WHEN Trim(Qualification) LIKE '%Master''s%' THEN 'PG'
            WHEN Trim(Qualification) ILIKE '%High School%' THEN 'High_School'
            ELSE Qualification 
        END AS Qualification
    FROM financial_risk_table
""")


In [21]:
df_qualification_altered.show()

+-------------+
|Qualification|
+-------------+
|          PhD|
|   Bachelor's|
|     Master's|
|   Bachelor's|
|   Bachelor's|
|          PhD|
|     Master's|
|   Bachelor's|
|   Bachelor's|
|   Bachelor's|
|     Master's|
|     Master's|
|          PhD|
|  High_School|
|  High_School|
|          PhD|
|  High_School|
|     Master's|
|   Bachelor's|
|          PhD|
+-------------+
only showing top 20 rows



In [22]:
from pyspark.sql.functions import when, col, lower, trim

df_qualification_altered = df_main_1.withColumn(
    "Qualification",
    when(lower(trim(col("Qualification"))).like("%bachelor's%"), "UG")
    .when(lower(trim(col("Qualification"))).like("%master's%"), "PG")
    .when(lower(trim(col("Qualification"))).like("%high school%"), "HS")
    .otherwise(col("Qualification"))
)

In [23]:
df_qualification_altered.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|           UG|       Widowed| 25605.0|       751.0|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|       3.0|      Samanthastad|  

In [24]:
spark.sql('''select case when income > 0 then 'amount' else 'null' end income_dist,count(1)
from financial_risk_table group by  income_dist ''').show()

+-----------+--------+
|income_dist|count(1)|
+-----------+--------+
|     amount|   12750|
|       null|    2250|
+-----------+--------+



In [25]:
# df_income = spark.sql(''' update financial_risk_table
#     set Income = 0.0 
#     where Income is NULL''').show()

df_income = df_qualification_altered.fillna(0.0, subset=['Income'])


In [26]:
df_income.show()
df_income.createOrReplaceTempView('financial_risk_table')

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|           UG|       Widowed| 25605.0|       751.0|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|       3.0|      Samanthastad|  

In [27]:
spark.sql("""
SELECT 
    CASE 
        WHEN Income IS NULL THEN 'missing'
        WHEN Income = 0 THEN 'zero'
        WHEN Income > 0 THEN 'amount'
        ELSE 'other'
    END AS income_dist,
    COUNT(1) AS total_count
FROM financial_risk_table
GROUP BY 
    CASE 
        WHEN Income IS NULL THEN 'missing'
        WHEN Income = 0 THEN 'zero'
        WHEN Income > 0 THEN 'amount'
        ELSE 'other'
    END

""").show()


+-----------+-----------+
|income_dist|total_count|
+-----------+-----------+
|       zero|       2250|
|     amount|      12750|
+-----------+-----------+



In [28]:
df_not_nulls = df_income.fillna(0.0, subset=['Loan_Amount','Credit_Score','Dependents','Previous_Defaults'])

In [29]:
df_not_nulls.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|           UG|       Widowed| 25605.0|       751.0|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|       3.0|      Samanthastad|  

In [30]:
df_main_final_1 = df_not_nulls \
    .withColumn("Dependents", col("Dependents").cast("int")) \
    .withColumn("Previous_Defaults", col("Previous_Defaults").cast("int"))

In [31]:
df_main_final_1.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|           UG|       Widowed| 25605.0|       751.0|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|         3|      Samanthastad|  

In [32]:
df_main_final_2 = df_main_final_1.withColumn('Credit_score',col('Credit_score').cast('int'))

In [33]:
df_main_final_2.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  1|      Male|           UG|       Widowed| 25605.0|         751|    14785.0|        Auto|       Unemployed|             17|           Fair| 0.33994669003424505|    285745.0|         3|      Samanthastad|  

In [34]:
df_main_final_2.filter(col("Income") == 0.0).count()
df_main_final_2.filter(col('Credit_score') == 0).count()
df_main_before_drop_junk = df_main_final_2

df_main_before_drop_junk.show(3)


+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|        City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------+-----+--------------------+-----------------+-----------+
|  1|Non-binary|           PG|        Single|     0.0|         651|    18653.0|        Home|    Self-employed|             16|           Good| 0.49222995000773917|    226806.0|         0| Curtisville|   PW|             Mayotte

In [35]:
df_dropped_junk = df_main_final_2.filter(
    (col("Income") != 0) & (col("Credit_score") != 0))

In [36]:
df_dropped_junk.show()

+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|  Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+--------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  2|    Female|           PG|       Widowed|102628.0|         654|    25442.0|        Home|       Unemployed|              8|           Poor|  0.1618325788225118|        NULL|         3|      West Patrick|  

In [37]:
from pyspark.sql.types import DecimalType

df_main_3 = df_dropped_junk.withColumn("Debt_to_Income_Ratio", col("Debt_to_Income_Ratio").cast(DecimalType(10, 3))) \
                            .withColumn('Income',col('Income').cast(DecimalType(10,3))) \
                            .withColumn('Loan_Amount',col('Loan_Amount').cast(DecimalType(10,3))) \
                            .withColumn('Assets_Value',col('Assets_Value').cast(DecimalType(10,3)))


In [38]:
df_main_3.show(14)

+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|    Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  2|    Female|           PG|       Widowed|102628.000|         654|  25442.000|        Home|       Unemployed|              8|           Poor|               0.162|        NULL|         3|      West Pa

In [39]:
df_final = df_main_3.fillna(0.000,subset = ['Assets_Value'])

In [40]:
df_final.show(9)

+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|    Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  2|    Female|           PG|       Widowed|102628.000|         654|  25442.000|        Home|       Unemployed|              8|           Poor|               0.162|       0.000|         3|      West Pa

In [41]:
df_final.write.mode("overwrite").parquet("Norm_financial_risk")

In [42]:
df_parquet_file = spark.read.parquet('Norm_financial_risk',header=True,inferschema=True)

In [44]:
df_parquet_file.show(9)

+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|    Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  2|    Female|           PG|       Widowed|102628.000|         654|  25442.000|        Home|       Unemployed|              8|           Poor|               0.162|       0.000|         3|      West Pa

In [51]:
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
hadoop_conf.set("fs.s3a.profile", "default")

df_final.write.mode("overwrite").parquet("s3a://testing-purpose-for-projects/sample/")

In [52]:
df = spark.read.parquet("s3a://testing-purpose-for-projects/sample/")
df.show(9)

+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
| id|    Gender|Qualification|Marital_status|    Income|Credit_score|Loan_Amount|Loan_Purpose|Employment_Status|Current_Job_YOE|Payment_History|Debt_to_Income_Ratio|Assets_Value|Dependents|              City|State|             Country|Previous_Defaults|Risk_Rating|
+---+----------+-------------+--------------+----------+------------+-----------+------------+-----------------+---------------+---------------+--------------------+------------+----------+------------------+-----+--------------------+-----------------+-----------+
|  2|    Female|           PG|       Widowed|102628.000|         654|  25442.000|        Home|       Unemployed|              8|           Poor|               0.162|       0.000|         3|      West Pa

In [45]:
print(spark._jvm.org.apache.hadoop.fs.s3a.S3AFileSystem)


In [46]:
print(spark.sparkContext._jvm.java.lang.System.getProperty("java.class.path"))


/usr/local/spark/conf/:/usr/local/spark/jars/ivy-2.5.1.jar:/usr/local/spark/jars/hadoop-yarn-server-web-proxy-3.3.4.jar:/usr/local/spark/jars/jakarta.validation-api-2.0.2.jar:/usr/local/spark/jars/arrow-vector-12.0.1.jar:/usr/local/spark/jars/spire_2.12-0.17.0.jar:/usr/local/spark/jars/leveldbjni-all-1.8.jar:/usr/local/spark/jars/shims-0.9.45.jar:/usr/local/spark/jars/log4j-core-2.20.0.jar:/usr/local/spark/jars/bonecp-0.8.0.RELEASE.jar:/usr/local/spark/jars/metrics-jvm-4.2.19.jar:/usr/local/spark/jars/kubernetes-model-rbac-6.7.2.jar:/usr/local/spark/jars/spark-mllib_2.12-3.5.0.jar:/usr/local/spark/jars/netty-resolver-4.1.96.Final.jar:/usr/local/spark/jars/spark-graphx_2.12-3.5.0.jar:/usr/local/spark/jars/flatbuffers-java-1.12.0.jar:/usr/local/spark/jars/commons-collections4-4.4.jar:/usr/local/spark/jars/hive-jdbc-2.3.9.jar:/usr/local/spark/jars/spark-mesos_2.12-3.5.0.jar:/usr/local/spark/jars/dropwizard-metrics-hadoop-metrics2-reporter-0.1.2.jar:/usr/local/spark/jars/paranamer-2.8.jar: